In [ ]:
import sys
sys.path.append("c:/Program Files/Stata18/utilities") #Importamos al ubicacion de Stata en el sistema

#cargamos stata y la configuración multiplataforma
from pystata import config
config.init('mp')

# 1. ESTADÍSTICA DESCRIPTIVA INICIAL

### 1.1. Configuración variables

In [ ]:
%%stata
use "MCVL18.dta", clear

*Eliminamos valores extremos salario
egen w99=pctile(s18), p(99.5) //Parte superior
egen w01=pctile(s18), p(00.5) //Parte inferior: SMI18=860
keep if s18>w01 & s18<w99

*Generamos variable fundamental
gen smi19=900
gen afecta=0
replace afecta=1 if s18<smi19*1.0125

*Variables simples: Homgeneizacion frente a EPA
**Sexo
rename sexo hombre
replace hombre=0 if hombre==2

**Edad
rename edad edadempresa //Creemos que es la edad empresa
gen edad=year-anonac+1
gen edad5 =.
replace edad5 = 1 if edad >= 16 & edad <= 19
replace edad5 = 2 if edad >= 20 & edad <= 24
replace edad5 = 3 if edad >= 25 & edad <= 29
replace edad5 = 4 if edad >= 30 & edad <= 34
replace edad5 = 5 if edad >= 35 & edad <= 39
replace edad5 = 6 if edad >= 40 & edad <= 44
replace edad5 = 7 if edad >= 45 & edad <= 49
replace edad5 = 8 if edad >= 50 & edad <= 54
replace edad5 = 9 if edad >= 55 & edad <= 59
replace edad5 = 10 if edad >= 60 & edad <= 64
label define e5 1 "16 a 19 anos" 2 "20 a 24 anos" 3 "25 a 29 anos" 4 "30 a 34 anos" 5 "35 a 39 anos" 6 "40 a 44 anos" 7 "45 a 49 anos" 8 "50 a 54 anos" 9 "55 a 59 anos" 10 "60 a 64 anos"
label values edad5 e5

**Zona geográfica
***Provincia
gen prov=provcuentacot2
label define prov_lbl 1 "Araba/Álava" 2 "Albacete" 3 "Alicante/Alacant" 4 "Almería" 5 "Ávila" ///
    6 "Badajoz" 7 "Balears, Illes" 8 "Barcelona" 9 "Burgos" 10 "Cáceres" 11 "Cádiz" 12 "Castellón /Castelló" ///
    13 "Ciudad Real" 14 "Córdoba" 15 "Coruña, A" 16 "Cuenca" 17 "Girona" 18 "Granada" 19 "Guadalajara" ///
    20 "Gipuzkoa" 21 "Huelva" 22 "Huesca" 23 "Jaén" 24 "León" 25 "Lleida" 26 "Rioja, La" 27 "Lugo" ///
    28 "Madrid" 29 "Málaga" 30 "Murcia" 31 "Navarra" 32 "Ourense" 33 "Asturias" 34 "Palencia" ///
    35 "Palmas, Las" 36 "Pontevedra" 37 "Salamanca" 38 "Santa Cruz de Tenerife" 39 "Cantabria" 40 "Segovia" ///
    41 "Sevilla" 42 "Soria" 43 "Tarragona" 44 "Teruel" 45 "Toledo" 46 "Valencia/València" 47 "Valladolid" ///
    48 "Bizkaia" 49 "Zamora" 50 "Zaragoza" 51 "Ceuta" 52 "Melilla"
label values prov prov_lbl

***Comunidad
gen comunidad = .
replace comunidad = 1 if inlist(prov, 1, 20, 48) // País Vasco
replace comunidad = 2 if inlist(prov, 2, 13, 16, 19, 45) // Castilla-La Mancha
replace comunidad = 3 if inlist(prov, 3, 12, 46) // Comunidad Valenciana
replace comunidad = 4 if inlist(prov, 4, 11, 14, 18, 21, 23, 29, 41) // Andalucía
replace comunidad = 5 if inlist(prov, 5, 9, 24, 34, 37, 40, 42, 47, 49) // Castilla y León
replace comunidad = 6 if inlist(prov, 6, 10) // Extremadura
replace comunidad = 7 if inlist(prov, 7) // Islas Baleares
replace comunidad = 8 if inlist(prov, 8, 17, 25, 43) // Cataluña
replace comunidad = 9 if inlist(prov, 15, 27, 32, 36) // Galicia
replace comunidad = 10 if inlist(prov, 28) // Comunidad de Madrid
replace comunidad = 11 if inlist(prov, 30) // Región de Murcia
replace comunidad = 12 if inlist(prov, 31) // Navarra
replace comunidad = 13 if inlist(prov, 33) // Asturias
replace comunidad = 14 if inlist(prov, 35, 38) // Canarias
replace comunidad = 15 if inlist(prov, 39) // Cantabria
replace comunidad = 16 if inlist(prov, 22, 44, 50) // Aragón
replace comunidad = 17 if inlist(prov, 26) // La Rioja
replace comunidad = 18 if inlist(prov, 51, 52) // Ceuta y Melilla

label define com 1 "Pais Vasco" 2 "Castilla-La Mancha" 3 "Comunidad Valenciana" 4 "Andalucia" 5 "Castilla y Leon" 6 "Extremadura" 7 "Islas Baleares" 8 "Cataluna" 9 "Galicia" 10 "Comunidad de Madrid" 11 "Region de Murcia" 12 "Navarra" 13 "Principado de Asturias" 14 "Islas Canarias" 15 "Cantabria" 16 "Aragon" 17 "La Rioja" 18 "Ceuta y Melilla"
label values comunidad com

*Variable pais de procedencia
gen paisnac = .
replace paisnac = 1 if paisnacim == 0
replace paisnac = 2 if inlist(paisnacim, 1, 10, 11, 15, 16, 19)
replace paisnac = 3 if inlist(paisnacim, 4, 14, 17, 20)
replace paisnac = 4 if inlist(paisnacim, 18, 21)
replace paisnac = 5 if inlist(paisnacim, 12, 23)
replace paisnac = 6 if paisnacim == 25
replace paisnac = 7 if inlist(paisnacim, 3, 6, 7, 8, 9, 13, 22, 26, 27)
replace paisnac = 8 if inlist(paisnacim, 5, 24)
label define pais 1 "Espana" 2 "UE-15" 3 "UE-27" 4 "Resto de Europa" 5 "africa" 6 "America del Norte" 7 "Centroamerica y Sudamerica" 8 "Asia y Oceania"
label values paisnac pais

*Extranjero
gen extranjero=0
replace extranjero=1 if paisnac!=1

*Nivel educativo
gen educ=.
replace educ=1 if inlist(niveleduca, 10, 11)
replace educ=2 if inlist(niveleduca, 20, 21, 22)
replace educ=3 if niveleduca==30
replace educ=4 if niveleduca==31
replace educ=5 if inlist(niveleduca, 40, 42, 32, 41, 43, 47)
replace educ=6 if inlist(niveleduca, 44, 45, 46, 48)
label define nivelest 1 "Sin estudios" 2 "Primaria Incompleta" 3 "Primaria" 4 "Secundaria" 5 "FP Media, Bachillerato" 6 "Educación Superior (FP, Univ.)"
label values educ nivelest

**Relacion laboral (meses)
gen anobaja2=anobaja
replace anobaja2=2018 if anobaja>2018
gen mesbaja2=mesbaja
replace mesbaja2=12 if anobaja>2018
gen relab=((anobaja2-anoalta)*12)+(mesbaja2-mesalta)

*Eliminamos valores atípicos
keep if edad5!=.
keep if paisnac!=.
keep if educ!=.

*Dividimos la base de datos
set seed 18563
gen byte train=(runiform()<=0.7)
label define tr 1 "Train" 0 "Test"
label values train tr

### 1.2. Distribución salarios

In [ ]:
%%stata
histogram s18, saving("1.Salario.gph", replace) xline(900) ///
	title({bf:GRÁFICO 2: DISTRIBUCIÓN SALARIAL (2018)}) xtitle("€/mes") ytitle("Frecuencia Relativa") ///
	xlabel(859 2300 3804) ylabel(0.0005 "5%" 0.001 "1%" 0.0015 "1.5%") ///
	note("Fuente: Muestra Continua de Vidas Laborales")
graph use 1.Salario.gph
graph export "1.Salario.png", as(png) replace

### 1.3. Tabla características sociodemográficas

In [ ]:
%%stata
**Dummies y continua
foreach x in hombre extranjero relab {
	qui ttest `x', by(afecta)
	scalar med`x'0=r(mu_1) //No afectado
	scalar sd`x'0=r(sd_1) 
	scalar med`x'1=r(mu_2) //Afectado
	scalar sd`x'1=r(sd_2) 
	scalar `x'diff=r(mu_2)-r(mu_1) //Diferencia de medias
	scalar `x'pvalue=r(p)
	matrix `x'= (med`x'1, med`x'0, `x'diff \ sd`x'1, sd`x'0, `x'pvalue)
}

matrix dumm=(hombre \ extranjero \ relab) //1 afectado 0 no afectado 
matrix colnames dumm= "Afectado (Media/ Desv. Est.)" ///
						"No afectado (Media/ Desv. Est.)" ///
						"Diferencia (Media/ P-valor)"
matrix rownames dumm= "Hombre" "-" ///
						"Extranjero" "-" ///
						"Antiguedad en la empresa" "-"			
						
**Edad
qui tab edad5, gen(gedad) //Creamos grupos de edad
foreach x in 1 2 3 4 5 6 7 8 9 10{
	qui ttest gedad`x', by(afecta)
	scalar medgedad`x'0=r(mu_1) //No afectado
	scalar sdgedad`x'0=r(sd_1)
	scalar medgedad`x'1=r(mu_2) //No afectado
	scalar sdgedad`x'1=r(sd_2)
	scalar gedad`x'diff=r(mu_2)-r(mu_1) //Diferencia de medias
	scalar gedad`x'pvalue=r(p)
	matrix edad`x'= (medgedad`x'1, medgedad`x'0, gedad`x'diff \ ///
						sdgedad`x'1, sdgedad`x'0, gedad`x'pvalue)
}

matrix edad=(edad1 \ edad2 \ edad3 \ edad4 \ edad5 \ edad6 \ edad7 \ edad8 \ edad9 \ edad10)
matrix colnames edad= "Afectado (Media/ Desv. Est.)" ///
						"No afectado (Media/ Desv. Est.)" ///
						"Diferencia (Media/ P-valor)"
matrix rownames edad = "16-19 años" "-" ///
                       "20-24 años" "-" ///
                       "25-29 años" "-" ///
                       "30-34 años" "-" ///
                       "35-39 años" "-" ///
                       "40-44 años" "-" ///
                       "45-49 años" "-" ///
                       "50-54 años" "-" ///
                       "55-59 años" "-" ///
                       "60-64 años" "-"
					   
**Nivel educativo
qui tab educ, gen(educ) //Creamos grupos
foreach x in 1 2 3 4 5 6 {
	qui ttest educ`x', by(afecta)
	scalar mededuc`x'0=r(mu_1) //No afectado
	scalar sdeduc`x'0=r(sd_1)
	scalar mededuc`x'1=r(mu_2) //No afectado
	scalar sdeduc`x'1=r(sd_2)
	scalar educ`x'diff=r(mu_2)-r(mu_1) //Diferencia de medias
	scalar educ`x'pvalue=r(p)
	matrix educ`x'= (mededuc`x'1, mededuc`x'0, educ`x'diff \ ///
						sdeduc`x'1, sdeduc`x'0, educ`x'pvalue)
}

matrix educ=(educ1 \ educ2 \ educ3 \ educ4 \ educ5 \ educ6)
matrix colnames educ= "Afectado (Media/ Desv. Est.)" ///
						"No afectado (Media/ Desv. Est.)" ///
						"Diferencia (Media/ P-valor)"
matrix rownames educ = "Sin estudios" "-" ///
                       "Primaria Incompleta" "-" ///
                       "Primaria" "-" ///
                       "Secundaria" "-" ///
                       "FP media, Bachillerato" "-" ///
                       "Educación superior" "-"

**Exportamos Resultados
***Afectados y no afectado
qui sum afecta
scalar N=r(N)
foreach x in 1 0 { 
	qui count if afecta==`x'
	scalar n`x'=r(N)
	scalar pct`x'=n`x'/N
}
matrix sum =(n1, n0, 0 \ pct1, pct0, 0)
matrix colnames sum= "Afectado (Media/ Desv. Est.)" ///
						"No afectado (Media/ Desv. Est.)" ///
						"Diferencia (Media/ P-valor)"
matrix rownames sum= "Obs" "Pct"

**Unimos matrices
matrix estdes=(dumm \ edad \ educ \ sum)
putexcel set 1.EstDes.xlsx, replace
putexcel A1=matrix(estdes), nformat(number_d2) names

# 2. GENERACIÓN MODELOS PREDICTIVOS

### 2.1. Configuración de las variables

In [ ]:
%%stata
*Generamos las variables necesarias para el modelo
**Edad
qui tab edad5, gen(ed)
global edad ed1 ed2 ed3 ed4 ed5 ed6 ed7 ed8 ed9
**Paisnac
qui tab paisnac, gen(pais)
global paisnac pais2 pais3 pais4 pais5 pais6 pais7 pais8
**Educación
qui tab educ, gen(ned)
global educ ned1 ned2 ned3 ned4 ned5
*Definimos con interacciones
global x2 1.hombre#(1.ed1 1.ed2 1.ed3 1.ed4 1.ed5 1.ed6 1.ed7 1.ed8 1.ed9 i.comunidad 1.pais2 1.pais3 1.pais4 1.pais5 1.pais6 1.pais7 1.pais8 1.ned1 1.ned2 1.ned3 1.ned4 1.ned5 c.relab) ///
	(1.ed1 1.ed2 1.ed3 1.ed4 1.ed5 1.ed6 1.ed7 1.ed8 1.ed9)#(i.comunidad 1.pais2 1.pais3 1.pais4 1.pais5 1.pais6 1.pais7 1.pais8 1.ned1 1.ned2 1.ned3 1.ned4 1.ned5 c.relab) ///
	i.comunidad#(1.pais2 1.pais3 1.pais4 1.pais5 1.pais6 1.pais7 1.pais8 1.ned1 1.ned2 1.ned3 1.ned4 1.ned5 c.relab) ///
	(1.pais2 1.pais3 1.pais4 1.pais5 1.pais6 1.pais7 1.pais8)#(1.ned1 1.ned2 1.ned3 1.ned4 1.ned5 c.relab) ///
	(1.ned1 1.ned2 1.ned3 1.ned4 1.ned5)#c.relab

### 2.2. Localización geográfica por comunidades

In [ ]:
%%stata
*Modelo logistico básico
logit afecta hombre $edad $paisnac $educ c.relab i.comunidad $x2  if train==1, difficult iterate(10)
est save 2.logitccaa.ster, replace

*Modelo logistico con regularización LASSO BIC
lasso logit afecta (hombre $edad $paisnac $educ c.relab) i.comunidad $x2  if train==1, selection(bic, gridminok) grid(60) stop(1e-3)
est save 2.bicccaa.ster, replace

*Modelo logistico con regularización LASSO CV
lasso logit afecta (hombre $edad $paisnac $educ c.relab) i.comunidad $x2  if train==1, selection(cv, gridminok) grid(55, ratio(1e-3)) stop(1e-4) tolerance(1e-5) 
est save 2.cvccaa.ster, replace

### 2.3. Localización geográfica por provincias

*Modelo logistico básico
logit afecta hombre $edad $paisnac $educ c.relab i.prov $x2  if train==1, difficult iterate(10)
est save 2.logitprov.ster,replace

*Modelo logistico con regularización LASSO BIC
lasso logit afecta (hombre $edad $paisnac $educ c.relab) i.prov $x2  if train==1, selection(bic, gridminok) grid(55) stop(1e-3)
est save 2.bicprov.ster, replace

*Modelo logistico con regularización LASSO CV
lasso logit afecta (hombre $edad $paisnac $educ c.relab) i.prov $x2  if train==1, selection(cv, gridminok) grid(50, ratio(1e-3)) stop(1e-4) tolerance(1e-5) 
est save 2.cvprov.ster, replace

# 3. EVALUACIÓN DE RESULTADOS

### 3.1. Definición punto de clasificación

El punto de clasificación representa la probabilidad estimada a partir de la cual definimos a un individuo como afectado o no por la reforma.

In [ ]:
%%stata
*Recurrimos al modelo logit
logit afecta hombre $edad $paisnac $educ c.relab i.prov $x2, difficult iterate(10)

*Filtramos a la base de datos de prueba en una base temporal
preserve
keep if train==0 //Nos movemos en la base de datos de prueba
 
*Proceso de maximización de sens y espec
**Guardamos valores 
lsens, nograph genprob(cutoff) gensens(sens) genspec(spec)
qui sum cutoff
scalar sd=r(sd)
**Buscamos el mejor cutoff
gen clasmed=(sens+spec)/2 //Clasificacion media
egen opt=pctile(clasmed), p(99.99) //Punto máximo de clasificacion media
***Cutoff óptimo
qui sum cutoff if clasmed>=opt
display "El cutoff es: " as result r(mean) 
***Cutoff óptimo para reducir sobre estimación
replace cutopt=cutoff+(0.22613*sd) if clasmed>=opt
qui sum cutopt
display "El cutoff óptimo es: " as result r(mean)

*Seleccion del cutoff optimo
lsens, saving("3.CutOffOpt.gph", replace) xline(0.1409) ///
	title({bf:GRÁFICO 3: Valores de Especificidad y Sensibilidad}) xtitle("Probabilidad de exposición a la reforma estimada (%)") ytitle("Individuos clasificados correctamente") ///
	xlabel(0 "0" 0.1409 "14,09" 0.25 "25" 0.50 "50" 0.75 "75" 1 "100") ///
	ylabel(0 "0%" 0.25 "25%" 0.50 "50%" 0.75 "75%" 1 "100%") ///
	legend(order(1 "Afectados (Sensibilidad)" 2 "No afectados(Especificidad)")) ///
	note("Fuente: Muestra Continua de Vidas Laborales")
graph use 3.CutOffOpt.gph
graph export "3.CutOffOpt.png", as(png) replace
restore

### 3.2. Gráfico distribuciones estimadas

In [ ]:
%%stata
*Filtramos a la base de datos de prueba
keep if train==0

In [ ]:
%%stata
*Bucle para cada distribución
foreach ef in ccaa prov {
	foreach mod in bic logit cv {
		histogram y`mod'`ef', frequency xline(0.1089) /// Probabilidad media de verse afectado
		title(Modelo `mod' con `ef') ///
		xtitle("Prob. verse afectado por subida") ///
		xlabel(0 "0" 0.1089 "11%" 0.3 "30%" 0.5 "50%" 0.75 "75%" 1 "100%") ///
		ylabel(5000 "5" 10000 "10" 15000 "15" 20000 "20" 25000 "25") ///
		ytitle("Número de personas (Miles)")
		
		graph save 3.Pred`mod'`ef'.gph, replace
	}
}

**Guardamos resumen de los modelos
graph combine 3.Predlogitccaa.gph 3.Predbicccaa.gph 3.Predcvccaa.gph 3.Predlogitprov.gph 3.Predbicprov.gph 3.Predcvprov.gph, ///
	title({bf:GRÁFICO 2: VALORES PREDICHOS (MCVL)})
graph export "3.PredResumen.png", as(png) replace

### 3.3. Tabla resultados predichos

%%stata
*VALORES PREDECIDOS
qui sum afecta
scalar mean_afecta = r(mean)
scalar sd_afecta = r(sd)
scalar min_afecta = r(min)
scalar max_afecta = r(max)
scalar obs_afecta = r(N)

*Modelos
foreach ef in ccaa prov {
	foreach mod in bic logit cv {
		qui sum y`mod'`ef'
		scalar mean_y`mod'`ef' = r(mean)
		scalar sd_y`mod'`ef' = r(sd)
		scalar min_y`mod'`ef' = r(min)
		scalar max_y`mod'`ef' = r(max)
		scalar obs_y`mod'`ef' = r(N)
	}
}

*Crear una matriz con los resultados
matrix pred = ( ///
    sd_afecta, sd_ybicccaa, sd_ylogitccaa, sd_ycvccaa, sd_ybicprov, sd_ylogitprov, sd_ycvprov\ /// 
    min_afecta, min_ybicccaa, min_ylogitccaa, min_ycvccaa, min_ybicprov, min_ylogitprov, min_ycvprov\ ///
    max_afecta, max_ybicccaa, max_ylogitccaa, max_ycvccaa, max_ybicprov, max_ylogitprov, max_ycvprov\ ///
    mean_afecta, mean_ybicccaa, mean_ylogitccaa, mean_ycvccaa, mean_ybicprov, mean_ylogitprov, mean_ycvprov\ ///
    obs_afecta, obs_ybicccaa, obs_ylogitccaa, obs_ycvccaa, obs_ybicprov, obs_ylogitprov, obs_ycvprov)
	
matrix colnames pred= "Observado" "Predicción BIC (CCAA)" "Predicción Logit (CCAA)" "Predicción CV (CCAA)" "Predicción BIC (Prov)" "Predicción Logit (Prov)" "Predicción CV (Prov)"
matrix rownames pred= "Std. Dev." "Min" "Max" "Mean" "Obs"

putexcel set 3.Resultados.xlsx, sheet(Predicciones, replace) replace
putexcel A1=matrix(pred), nformat(number_d2) names

### 3.4. Medidas de desvianza

In [ ]:
%%stata

*Resumen de los modelos:
lassogof bicccaa logitccaa cvccaa bicprov logitprov cvprov, over(train)
*Exportamos resultados
matrix lasogof=r(table)'
matrix colnames lasogof="Lasso BIC (CCAA)" "Logit (CCAA)" "Lasso CV (CCAA)" "Lasso BIC (Prov)" "Logit (Prov)" "Lasso CV (Prov)"
matrix rownames lasogof="Desviance" "Desv Ratio" "N Obs"

putexcel set 3.Resultados.xlsx, sheet(Lassogof, replace) modify
putexcel A1=matrix(lasogof), nformat(number_d2) names

### 3.5. Matriz de confusión

In [ ]:
%%stata

foreach ef in ccaa prov{
*EFECTOS FIJOS `ef'
	foreach mod in bic logit cv{
		*Predicciones
		gen pafecta_`mod'`ef' = 0
		replace pafecta_`mod'`ef' = 1 if y`mod'`ef' >=0.140863595 //Media de los valores hallados en 3.CutOff.do

		*Tabla
		count if afecta == 1 & pafecta_`mod'`ef' == 1
		scalar vp_`mod'`ef' = r(N) //Verdaderos positivos
		count if afecta == 0 & pafecta_`mod'`ef' == 1
		scalar fn_`mod'`ef' = r(N) // Falsos Negativos
		count if pafecta_`mod'`ef' == 1
		scalar tot_pred1`mod'`ef' = r(N) // Total Predecidos Positivos
		count if afecta == 1 & pafecta_`mod'`ef' == 0
		scalar fp_`mod'`ef' = r(N) // Falsos positivos
		count if afecta == 0 & pafecta_`mod'`ef' == 0
		scalar vn_`mod'`ef' = r(N) // Verdaderos Negativos
		count if pafecta_`mod'`ef' == 0
		scalar tot_pred0`mod'`ef' = r(N) // Total Predecidos Negativos
		count if afecta == 1
		scalar tot_obs1`mod'`ef' = r(N) // Total Afectados
		count if afecta == 0
		scalar tot_obs0`mod'`ef' = r(N) // Total no afectados
		scalar tot_`mod'`ef' = tot_obs1`mod'`ef' + tot_obs0`mod'`ef' // Total
		**Creamos Matriz
		matrix tabla`mod'`ef' = J(3, 3, .)
		matrix tabla`mod'`ef' = (vp_`mod'`ef', fn_`mod'`ef', tot_pred1`mod'`ef' \ ///
							 fp_`mod'`ef', vn_`mod'`ef', tot_pred0`mod'`ef' \ ///
							 tot_obs1`mod'`ef', tot_obs0`mod'`ef', tot_`mod'`ef')

		**Titulos
		matrix colnames tabla`mod'`ef' = "Afectado Observado D" ///
										 "No afectado Observado ~D" ///
										 "Total Predecidos"
										 
		matrix rownames tabla`mod'`ef' = "Afectado Predecido +" ///
										 "No afectado Predecido -" ///
										 "Total Observados"
					
		*Porcentajes
		matrix pct`mod'`ef' = J(9, 1, .)
		matrix pct`mod'`ef' = (vp_`mod'`ef' / tot_obs1`mod'`ef'*100\ ///
						   vn_`mod'`ef' / tot_obs0`mod'`ef'*100\ ///
						   vp_`mod'`ef' / tot_pred1`mod'`ef'*100\ ///
						   vn_`mod'`ef' / tot_pred0`mod'`ef'*100\ ///
						   (vp_`mod'`ef'+ vn_`mod'`ef') / tot_`mod'`ef'*100)
		**Titulos
		matrix colnames pct`mod'`ef'= `mod'`ef' 
		matrix rownames pct`mod'`ef' = "Sensib +/D" ///
									"Espec -/~D" ///
									"VPP D/+" ///
									"VPN ~D/-" ///
									"Correct_Clasif"
	}

	matrix confusion1`ef'=(pctbic`ef', pctlogit`ef',pctcv`ef')
	putexcel set 3.Resultados.xlsx, sheet(PctConfusion`ef', replace) modify
	putexcel A1 = matrix(confusion1`ef'), nformat(number_d2) names
	matrix confusion2`ef'=(tablabic`ef', tablalogit`ef',tablacv`ef')
	putexcel set 3.Resultados.xlsx, sheet(TablaConfusion`ef', replace) modify
	putexcel A1 = matrix(confusion2`ef'), names
}